# Lesson 2: TTRL with Real LLMs (Naive Voting)

In this lesson, we use the local **Ollama** library to perform Test-Time Scaling on a distinct math problem.

**Goal**: Improve accuracy on "Is 91 a prime number?" using Majority Voting.

### Prerequisites
*   `pip install ollama`
*   `ollama serve` running.
*   `ollama pull mistral:7b` done.

In [ ]:
import time
from collections import Counter
from rich.console import Console
try:
    import ollama
except ImportError:
    print("Please run: pip install ollama")

console = Console()

# --- CONFIGURATION ---
MODEL_NAME = "mistral:7b"
console.print(f"[bold]Connecting to Local Model:[/bold] {MODEL_NAME}")

In [ ]:
def get_completion(prompt: str, temp: float = 0.7) -> str:
    """Gets a single completion from Ollama."""
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "You are a math assistant. Answer with just the result. Is the number Prime? Yes/No."},
                {"role": "user", "content": prompt}
            ],
            options={"temperature": temp}
        )
        return response['message']['content'].strip()
    except Exception as e:
        return f"Error: {e}"

## 1. Zero-Shot Attempt
We ask about 91 ($7 \times 13$). Many models fail here.

In [ ]:
PROBLEM = "Is 91 a prime number? Answer Yes or No."
console.print(f"\n[bold cyan]Problem:[/bold cyan] {PROBLEM}")

# Single Attempt (Greedy decoding)
start_t = time.time()
greedy_ans = get_completion(PROBLEM, temp=0.0)
console.print(f"[bold]Greedy Answer (Temp=0):[/bold] {greedy_ans} (took {time.time()-start_t:.2f}s)")

## 2. TTRL Loop: Sample & Vote

In [ ]:
N_SAMPLES = 5
console.print(f"\n[bold yellow]Running TTRL Loop (N={N_SAMPLES})...[/bold yellow]")

samples = []
for i in range(N_SAMPLES):
    ans = get_completion(PROBLEM, temp=0.9) # High temp for diversity
    # Simple normalization to extract Yes/No
    clean_ans = "Yes" if "yes" in ans.lower() else "No" if "no" in ans.lower() else "Unsure"
    samples.append(clean_ans)
    console.print(f"  Sample {i+1}: {ans[:50].replace('\n', ' ')}... -> [blue]{clean_ans}[/blue]")

## 3. Majority Consensus

In [ ]:
counts = Counter(samples)
consensus_ans, votes = counts.most_common(1)[0]

console.print(f"\n[bold]Final Consensus:[/bold] [green]{consensus_ans}[/green] ({votes}/{N_SAMPLES} votes)")

GROUND_TRUTH = "No" # 91 is not prime
if consensus_ans == GROUND_TRUTH:
    console.print("✅ [bold green]CORRECT[/bold green] - TTRL fixed the error!")
else:
    console.print(f"❌ [bold red]FAIL[/bold red] - Even TTRL failed. (Truth: {GROUND_TRUTH})")